# Table of Contents
 <p><div class="lev1"><a href="#Initialization-Cells"><span class="toc-item-num">1 - </span>Initialization Cells</a></div><div class="lev1"><a href="#Gather-Data-from-Big-Query-GDELT"><span class="toc-item-num">2 - </span>Gather Data from Big Query GDELT</a></div><div class="lev2"><a href="#Original-Queries"><span class="toc-item-num">2.1 - </span>Original Queries</a></div><div class="lev1"><a href="#New-Queries---Multiple-Years-and-Quarter"><span class="toc-item-num">3 - </span>New Queries - Multiple Years and Quarter</a></div>

# Initialization Cells

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:

import json
import os
import pandas as pd
from shapely.geometry import shape, Point

In [3]:
from geo import geo

# Gather Data from Big Query GDELT

In [4]:
# Imports the Google Cloud client library
from google.cloud import bigquery
from pandas.io import gbq
import pandas as pd


#gbq._authenticate = lambda: client
# Instantiates a client
bigquery_client = bigquery.Client()


## Original Queries


In [76]:
# political event counts and average 
query_pol = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND SQLDATE>20170601 AND SQLDATE<20170901
AND ActionGeo_CountryCode='IR'
AND (EventCode='1721' OR EventCode='1722' OR EventCode='1723' OR EventCode='1724' OR EventBaseCode='173' OR EventBaseCode='174' OR EventBaseCode='182' OR EventBaseCode='183')
GROUP BY ActionGeo_ADM1Code, 
ORDER BY ActionGeo_ADM1Code
"""

In [103]:
# ESG event counts and average
query_esg = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND SQLDATE>20170101 AND SQLDATE<20170901
AND ActionGeo_CountryCode='IR'
AND (Actor1Type1Code='ENV' OR Actor1Type2Code= 'ENV' OR Actor1Type3Code='ENV' 
OR Actor1Type1Code='LAB' OR Actor1Type2Code= 'LAB' OR Actor1Type3Code='LAB' OR 
Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='NGOIHF')
GROUP BY ActionGeo_ADM1Code
ORDER BY ActionGeo_ADM1Code
"""

# New Queries - Multiple Years and Quarter

In [14]:
# political event counts and average 
query_pol = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code, Quarter(DATE(CAST(SQLDATE as STRING))) as q, Year(DATE(CAST(SQLDATE as STRING))) as y
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND  Year > 2014
AND ActionGeo_CountryCode='IR'
AND (EventCode='1721' OR EventCode='1722' OR EventCode='1723' OR EventCode='1724' OR EventBaseCode='173' OR EventBaseCode='174' OR EventBaseCode='182' OR EventBaseCode='183')
GROUP BY ActionGeo_ADM1Code,y,q
ORDER BY ActionGeo_ADM1Code, y, q
"""

In [15]:
# ESG event counts and average
query_esg = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code, Quarter(DATE(CAST(SQLDATE as STRING))) as q, Year(DATE(CAST(SQLDATE as STRING))) as y
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND Year > 2014
AND ActionGeo_CountryCode='IR'
AND (Actor1Type1Code='ENV' OR Actor1Type2Code= 'ENV' OR Actor1Type3Code='ENV' 
OR Actor1Type1Code='LAB' OR Actor1Type2Code= 'LAB' OR Actor1Type3Code='LAB' OR 
Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='NGOIHF')
GROUP BY ActionGeo_ADM1Code,y,q
ORDER BY ActionGeo_ADM1Code, y, q
"""

## Execute Queries

In [16]:
results_pol = gbq.read_gbq(query_pol, project_id='expedition-hacks-morocco')
results_pol.columns = ['pol count', 'pol average', 'Adm1Code', 'quarter', 'year']
results_esg = gbq.read_gbq(query_esg, project_id='expedition-hacks-morocco')
results_esg.columns = ['esg count', 'esg average', 'Adm1Code','quarter', 'year']


URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json
Making request: POST https://accounts.google.com/o/oauth2/token


Requesting query... 

URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json


ok.
Job ID: job_XXqJDJgkDekKRE1y2Q13ujrT9Ie3
Query running...


URL being requested: GET https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/queries/job_XXqJDJgkDekKRE1y2Q13ujrT9Ie3?alt=json


...

Requesting query... 

URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json


ok.
Job ID: job_KsCf-FQiGjxGtVlP5Ds2UMhQ3pKn
Query running...


URL being requested: GET https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/queries/job_KsCf-FQiGjxGtVlP5Ds2UMhQ3pKn?alt=json


Query done.
Processed: 9.5 GB
Standard price: $0.05 USD

Retrieving results...
Got 341 rows.

Total time taken 1.75 s.
Finished at 2017-10-14 02:14:58.


In [24]:
results_pol.to_csv('iran_results_pol')
results_esg.to_csv('iran_results_esg')

In [22]:
results_pol.head()

,pol count,pol average,Adm1Code,quarter,year
0,1186,22.807692,IR,1,2015
1,2549,28.010989,IR,2,2015
2,2725,29.619565,IR,3,2015
3,2557,27.793478,IR,4,2015
4,3557,39.087912,IR,1,2016


In [23]:
results_esg.head()

,esg count,esg average,Adm1Code,quarter,year
0,132,3.473684,IR,1,2015
1,230,3.150685,IR,2,2015
2,342,3.931034,IR,3,2015
3,235,2.901235,IR,4,2015
4,325,3.869048,IR,1,2016
